In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()
"""
Issues Encountered:
cant have negative numbers in multidimensional arrays
cant have camel case words
cant have float attributes
"""

Note: you may need to restart the kernel to use updated packages.


'\nIssues Encountered:\ncant have negative numbers in multidimensional arrays\ncant have camel case words\ncant have float attributes\n'

In [2]:
### Set up Yolo UDF
cursor.query("""
            CREATE UDF IF NOT EXISTS Yolo
            TYPE  ultralytics
            'model' 'yolov8m.pt';
    """).df() 

,0
0,"UDF Yolo already exists, nothing added."


In [3]:
### Set up Monodepth UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS MonodepthDetection
            IMPL'/home/youse/apperception/eva/udfs/monodepth_detection.py';
    """).df()

,0
0,"UDF MonodepthDetection already exists, nothing..."


In [4]:
### Set up Location UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS LocationDetection
            IMPL'/home/youse/apperception/eva/udfs/location_detection.py';
    """).df()

,0
0,"UDF LocationDetection already exists, nothing ..."


In [5]:
### Set up Intersection Query UDF
cursor.query(""" 
            CREATE UDF IF NOT EXISTS IntersectionQuery
            IMPL'/home/youse/apperception/eva/udfs/intersection_query.py';
    """).df()

,0
0,"UDF IntersectionQuery already exists, nothing ..."


In [6]:
### Load video
video_name = "boston-seaport-scene-0757-CAM_FRONT.mp4"
scene = "scene-0757"
video_path = "/data/processed/full-dataset/mini/videos/"
cursor.query("DROP TABLE IF EXISTS ObjectDetectionVideos;").df()
cursor.load(file_regex=video_path + video_name, format="VIDEO", table_name='ObjectDetectionVideos').df()

,0
0,Number of loaded VIDEO: 1


In [7]:
cursor.query("DROP TABLE IF EXISTS CameraConfigs;").df()
cursor.create_table("CameraConfigs", if_not_exists=True, columns="""
                cameraid TEXT(15),
                framenum INTEGER,
                cameratranslation NDARRAY FLOAT32(ANYDIM),
                camerarotation TEXT(100),
                cameraintrinsic NDARRAY FLOAT32(ANYDIM),
                egoheading INTEGER
            """).df()


""


In [8]:
### Add Camera Configs
%cd ..
from apperception.database import database
import pandas as pd


result = database.execute(f"SELECT cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading FROM Cameras WHERE cameraId = '{scene}'")
df = pd.DataFrame()
for r in result:
    cameraId, frameNum, cameraTranslation, cameraRotation, cameraIntrinsic, egoHeading = r
    cameraTranslation = list(cameraTranslation)
    cursor.query(f"""INSERT INTO CameraConfigs (cameraid, framenum, cameratranslation, camerarotation, cameraintrinsic, egoheading) VALUES
                                ('{cameraId}', {frameNum}, {cameraTranslation}, '{cameraRotation}', {cameraIntrinsic}, {egoHeading});""").df()
df
    

/home/youse/apperception


""


In [9]:
cursor.query("SELECT id, egoheading FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum").df()

,objectdetectionvideos.id,cameraconfigs.egoheading
0,1,63.418377
1,2,63.410492
2,3,63.406160
3,4,63.397694
4,5,63.397503
5,6,63.394688
6,7,63.392323
7,8,63.392437
8,9,63.391420


In [10]:
cursor.query("SELECT id, MonodepthDetection(data) FROM ObjectDetectionVideos").df()

/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/youse/miniconda3/envs/apperception/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


CUDA is available.
 > 0: Tesla T4
                                               depth
0  [[[12.782349586486816, 12.782535552978516, 12....
1  [[[12.378500938415527, 12.38070297241211, 12.3...
2  [[[12.426396369934082, 12.423896789550781, 12....
3  [[[13.870692253112793, 13.869702339172363, 13....
4  [[[13.010868072509766, 13.015649795532227, 13....
5  [[[11.58127212524414, 11.590614318847656, 11.6...
6  [[[9.93977165222168, 9.939776420593262, 9.9397...
                                               depth
0  [[[13.995400428771973, 13.994840621948242, 13....
1  [[[13.258557319641113, 13.255433082580566, 13....
2  [[[13.249404907226562, 13.248727798461914, 13....
3  [[[14.358054161071777, 14.36323356628418, 14.3...
4  [[[15.38269329071045, 15.38613510131836, 15.39...
5  [[[15.49009895324707, 15.493023872375488, 15.5...
6  [[[15.70239543914795, 15.705099105834961, 15.7...
                                               depth
0  [[[16.33663558959961, 16.340192794799805, 16.3...
1  [[[14.876

,objectdetectionvideos.id
0,0
1,1
2,2
3,3
4,4
...,...
232,232
233,233
234,234
235,235


In [11]:
### Run query
cursor.query("""
            SELECT framenum, cameraid FROM ObjectDetectionVideos JOIN CameraConfigs ON id = framenum
            WHERE IntersectionQuery(LocationDetection(Yolo(data), MonodepthDetection(data), cameratranslation, camerarotation, cameraintrinsic))
""").df()


KeyboardInterrupt: 